# TCG Scanner - Model Conversion to ONNX

This notebook converts trained PyTorch models to ONNX format for Flutter mobile deployment.

**Models to convert:**
1. YOLOv8 Detection Model (best.pt) → detection.onnx
2. FastViT Embedding Model (embedding_model.pt) → embedding.onnx

**Why ONNX instead of TFLite:**
- ✅ Better YOLOv8 performance on mobile
- ✅ Single runtime for both models (onnxruntime)
- ✅ No TensorFlow dependency issues
- ✅ Excellent Flutter support

**Requirements:**
- Trained models in Google Drive
- No GPU needed

## 1. Setup

In [ ]:
# Install required packages
print("Installing packages...")

# Core packages (no TensorFlow needed!)
!pip install -q ultralytics>=8.0.0
!pip install -q timm>=0.9.12
!pip install -q onnx>=1.14.0
!pip install -q onnxruntime>=1.16.0

print("✅ All packages installed!")
print("\n📦 Export Format: ONNX for both models")
print("   - Detection: YOLOv8 → ONNX")
print("   - Embedding: FastViT → ONNX")
print("   - Runtime: onnxruntime-flutter")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
from pathlib import Path

# Set paths
DRIVE_ROOT = Path('/content/drive/MyDrive/tcg-scanner')
MODELS_DIR = DRIVE_ROOT / 'models'
OUTPUT_DIR = DRIVE_ROOT / 'models/tflite'

# Create output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Drive root: {DRIVE_ROOT}")
print(f"Models dir: {MODELS_DIR}")
print(f"Output dir: {OUTPUT_DIR}")

In [ ]:
# Verify model files exist
detection_model_path = MODELS_DIR / 'detection/best.pt'
embedding_model_path = MODELS_DIR / 'embedding/embedding_model.pt'

print("Checking model files...")
print(f"Detection model: {detection_model_path.exists()} - {detection_model_path}")
print(f"Embedding model: {embedding_model_path.exists()} - {embedding_model_path}")

if not detection_model_path.exists():
    print("\nWARNING: Detection model not found!")
    print("Expected path: models/detection/best.pt")
    
if not embedding_model_path.exists():
    print("\nWARNING: Embedding model not found!")
    print("Expected path: models/embedding/embedding_model.pt")

## 2. Convert Detection Model (YOLOv8)

In [ ]:
from ultralytics import YOLO

print("="*50)
print("Converting Detection Model (YOLOv8)")
print("="*50)

# Load the trained model
print(f"\nLoading model from: {detection_model_path}")
yolo_model = YOLO(str(detection_model_path))

# Export to ONNX (better mobile support than TFLite for YOLO)
print("\nExporting to ONNX (recommended for mobile YOLO)...")
export_path = yolo_model.export(format='onnx', imgsz=640)

print(f"\n✅ Export successful: {export_path}")
print(f"   Format: ONNX (use onnxruntime-flutter)")
print(f"\n💡 ONNX is recommended for YOLOv8 on mobile:")
print(f"   - Better performance than TFLite for YOLO")
print(f"   - Official ONNX Runtime support in Flutter")
print(f"   - No TensorFlow compatibility issues")

In [ ]:
# Copy ONNX model to output directory
import shutil

detection_output = OUTPUT_DIR / 'detection.onnx'

if export_path:
    shutil.copy(str(export_path), str(detection_output))
    size_mb = detection_output.stat().st_size / 1024 / 1024
    print(f"✅ Detection model saved to: {detection_output}")
    print(f"   Size: {size_mb:.2f} MB")
    print(f"\n📦 Flutter package: onnxruntime")
    print(f"   Add to pubspec.yaml: onnxruntime: ^1.14.0")
else:
    print("❌ ERROR: Export failed")

<cell_type>markdown</cell_type>## 3. Convert Embedding Model (FastViT)

**Method**: Using ai-edge-torch (Google's official PyTorch→TFLite converter)

This is the modern, recommended approach for PyTorch to TFLite conversion.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

print("="*50)
print("Converting Embedding Model (FastViT)")
print("="*50)

# Define the model architecture (MUST match training exactly!)
class EmbeddingModel(nn.Module):
    def __init__(self, backbone='fastvit_t12', embedding_dim=384, dropout=0.2):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=False, num_classes=0)

        # Get feature dimension
        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224)
            features = self.backbone(dummy)
            feature_dim = features.shape[-1]

        # Must match training architecture exactly (includes Dropout!)
        self.head = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, embedding_dim),
        )

    def forward(self, x):
        features = self.backbone(x)
        embeddings = self.head(features)
        return F.normalize(embeddings, p=2, dim=1)

# Load trained weights
print(f"\nLoading model from: {embedding_model_path}")
embed_model = EmbeddingModel()
state_dict = torch.load(embedding_model_path, map_location='cpu')
embed_model.load_state_dict(state_dict)
embed_model.eval()

print("Model loaded successfully!")

In [ ]:
# Export Embedding Model to ONNX
import torch

print("="*50)
print("Converting Embedding Model to ONNX")
print("="*50)

# Prepare sample input
sample_input = torch.randn(1, 3, 224, 224)

# Export to ONNX
onnx_output = OUTPUT_DIR / 'embedding.onnx'

print("\nExporting to ONNX...")
torch.onnx.export(
    embed_model,
    sample_input,
    str(onnx_output),
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['embedding'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'embedding': {0: 'batch_size'}
    }
)

size_mb = onnx_output.stat().st_size / 1024 / 1024
print(f"\n✅ Embedding model saved to: {onnx_output}")
print(f"   Size: {size_mb:.2f} MB")
print(f"   Format: ONNX")

# Test if normalized
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession(str(onnx_output))
test_input = np.random.randn(1, 3, 224, 224).astype(np.float32)
output = session.run(None, {'input': test_input})[0]
norm = np.linalg.norm(output[0])

print(f"\n   Test embedding norm: {norm:.4f}")
if 0.95 < norm < 1.05:
    print(f"   ✅ L2 normalized (norm ~1.0)")
else:
    print(f"   ⚠️  Not normalized - add normalization in Flutter")
    print(f"      embedding = embedding / ||embedding||")

<cell_type>markdown</cell_type>## 4. Verify Converted Models & Flutter Integration

In [ ]:
import onnxruntime as ort
import numpy as np

print("="*60)
print("CONVERSION SUMMARY")
print("="*60)

print(f"\nOutput directory: {OUTPUT_DIR}")
print("\nGenerated ONNX models:")

for f in OUTPUT_DIR.glob('*.onnx'):
    size_mb = f.stat().st_size / 1024 / 1024
    print(f"  ✅ {f.name}: {size_mb:.2f} MB")

print("\n" + "="*60)
print("FLUTTER INTEGRATION")
print("="*60)

print("""
1. Add to pubspec.yaml:
   dependencies:
     onnxruntime: ^1.14.0

2. Copy models to Flutter:
   - assets/models/detection.onnx
   - assets/models/embedding.onnx

3. Load models in Flutter:
   ```dart
   import 'package:onnxruntime/onnxruntime.dart';
   
   final detectionSession = OrtSession.fromAsset(
     'assets/models/detection.onnx'
   );
   
   final embeddingSession = OrtSession.fromAsset(
     'assets/models/embedding.onnx'
   );
   ```

4. Run inference:
   ```dart
   // Detection
   final detections = await detectionSession.run([imageTensor]);
   
   // Embedding
   final embedding = await embeddingSession.run([cardTensor]);
   ```
""")

print("="*60)
print("✅ CONVERSION COMPLETE!")
print("="*60)